In [1]:
!pip install numpy
!pip install pandas
!pip install pot
!pip install anndata
!pip install scanpy
!pip install scipy

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import ot
import sys
import anndata
import scanpy as sc
from scipy.sparse import issparse
from scipy.sparse import csr_matrix

[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [5]:
adata = anndata.read_h5ad("/workspace/ImputationOT/data/10x-Multiome-Pbmc10k-ATAC.h5ad")
adata.var_names_make_unique

<bound method AnnData.var_names_make_unique of AnnData object with n_obs × n_vars = 9631 × 107194
    obs: 'domain', 'protocol', 'dataset', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_ATAC', 'nFeature_ATAC', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'ATAC.weight', 'wsnn_res.0.8', 'seurat_clusters', 'sub.cluster', 'cell_type'
    var: 'feature_types', 'genome', 'chrom', 'chromStart', 'chromEnd', 'n_counts'>

In [6]:
print(adata.obs['cell_type'].unique())

['CD4 TCM', 'CD4 Naive', 'CD8 Naive', 'CD16 Mono', 'NK', ..., 'gdT', 'CD8 TEM_2', 'pDC', 'Intermediate B', 'HSPC']
Length: 19
Categories (19, object): ['CD4 Naive', 'CD4 TCM', 'CD4 TEM', 'CD8 Naive', ..., 'Treg', 'cDC', 'gdT', 'pDC']


In [8]:
## preprocess1
adata_GEX1 = adata[:, adata.var['feature_types'] == 'GEX'].copy()
adata_ADT1 = adata[:, adata.var['feature_types'] == 'ADT'].copy()
### step 1
sc.pp.normalize_total(adata_GEX1, target_sum=1e4)
sc.pp.normalize_total(adata_ADT1, target_sum=1e4)
adata1 = adata.copy()
adata1.X[:, adata.var['feature_types'] == 'GEX'] = adata_GEX1.X
adata1.X[:, adata.var['feature_types'] == 'ADT'] = adata_ADT1.X
### step 2
sc.pp.log1p(adata1)
### step 3
adata_GEX1 = adata1[:, adata.var['feature_types'] == 'GEX'].copy()
adata_ADT1 = adata1[:, adata.var['feature_types'] == 'ADT'].copy()
sc.pp.highly_variable_genes(
    adata_GEX1,
    n_top_genes=2000
)
print(adata_GEX1.var['highly_variable'])
print(adata1.var['feature_types'] == 'ADT')
print()

adata1 = adata1[:, (adata1.var['feature_types'] == 'ADT') | (adata_GEX1.var['highly_variable'])]
print(adata1)
print()

AL627309.5    False
LINC01409     False
LINC01128     False
LINC00115      True
FAM41C        False
              ...  
MT-CYB        False
AC011043.1    False
AL592183.1    False
AC240274.1    False
AC007325.4    False
Name: highly_variable, Length: 13953, dtype: bool
AL627309.5    False
LINC01409     False
LINC01128     False
LINC00115     False
FAM41C        False
              ...  
HLA-E          True
CD82           True
CD101          True
CD88           True
CD224          True
Name: feature_types, Length: 14087, dtype: bool

View of AnnData object with n_obs × n_vars = 90261 × 2134
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [15]:
X = adata.X
X = X[:, :13953]
print("GEX data")
print("Matrix Shape:", X.shape)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

X = adata.X
X = X[:, 13953:]
print("ADT data")
print("Matrix Shape:", X.shape)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())
print()

### preprocess2
adata_GEX2 = adata[:, adata.var["feature_types"] == "GEX"].copy()
adata_ADT2 = adata[:, adata.var["feature_types"] == "ADT"].copy()
### step 1
sc.pp.normalize_total(adata_GEX2, target_sum=1e4)
sc.pp.normalize_total(adata_ADT2, target_sum=1e4)
### step 2
sc.pp.log1p(adata_GEX2)
sc.pp.log1p(adata_ADT2)
### step 3
sc.pp.highly_variable_genes(
    adata_GEX2,
    n_top_genes=2000,
    subset=True
)
adata2 = anndata.concat([adata_GEX2, adata_ADT2], axis=1, merge="same")   # X(:,1): GEX, X(:,2): ADT

X = adata2.X
X = X[:, :2000]
print("GEX data")
print("Matrix Shape:", X.shape)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

X = adata2.X
X = X[:, 2000:]
print("ADT data")
print("Matrix Shape:", X.shape)
print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

GEX data
Matrix Shape: (90261, 13953)
Density: 0.10429761495639489
Minimum Value: 0.0
Maximum Value: 21078940.0
ADT data
Matrix Shape: (90261, 134)
Density: 0.8384980405910752
Minimum Value: 0.0
Maximum Value: 7.910906



/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


GEX data
Matrix Shape: (90261, 2000)
Density: 0.0811086681955662
Minimum Value: 0.0
Maximum Value: 9.116693
ADT data
Matrix Shape: (90261, 134)
Density: 0.8384980405910752
Minimum Value: 0.0
Maximum Value: 7.1100817


In [12]:
# print(compare_sparse_anndata_X(adata_GEX1, adata_GEX2))
# print(compare_sparse_anndata_X(adata_ADT1, adata_ADT2))
print(compare_sparse_anndata_X(adata1, adata2))

False
